In [182]:
!pip install gensim
!pip install matplotlib
!pip install -U sklearn

#  Graph  Construction

In [183]:
import networkx as nx
import random
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx

In [184]:
dataset = KarateClub()[0]

In [185]:
dataset

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [186]:
dataset.y

tensor([1, 1, 1, 1, 3, 3, 3, 1, 0, 1, 3, 1, 1, 1, 0, 0, 3, 1, 0, 1, 0, 1, 0, 0,
        2, 2, 0, 0, 2, 0, 0, 2, 0, 0])

In [187]:
karate_graph = to_networkx(dataset)

for node in karate_graph.nodes:
    karate_graph.nodes[node]["class"] = dataset.y.tolist()[node]
    

In [188]:
karate_graph.nodes(data=True)

NodeDataView({0: {'class': 1}, 1: {'class': 1}, 2: {'class': 1}, 3: {'class': 1}, 4: {'class': 3}, 5: {'class': 3}, 6: {'class': 3}, 7: {'class': 1}, 8: {'class': 0}, 9: {'class': 1}, 10: {'class': 3}, 11: {'class': 1}, 12: {'class': 1}, 13: {'class': 1}, 14: {'class': 0}, 15: {'class': 0}, 16: {'class': 3}, 17: {'class': 1}, 18: {'class': 0}, 19: {'class': 1}, 20: {'class': 0}, 21: {'class': 1}, 22: {'class': 0}, 23: {'class': 0}, 24: {'class': 2}, 25: {'class': 2}, 26: {'class': 0}, 27: {'class': 0}, 28: {'class': 2}, 29: {'class': 0}, 30: {'class': 0}, 31: {'class': 2}, 32: {'class': 0}, 33: {'class': 0}})

# Random Walk

In [189]:
'''
    graph : Graph
    start_node : start_node of random walk sequence
    num_steps : the length of random walk sequnce
'''
def random_walk(graph, start_node, num_steps):
    result = []
    current_node = start_node
    for _ in range(num_steps):
        result.append(current_node)
        neighbors = list(graph .neighbors(current_node))
        if not neighbors:
            break
        current_node = random.choice(neighbors)

    return result

In [190]:
start_node = random.choice(list(karate_graph.nodes()))
num_steps = 10  # the number of steps for random walk (length of sequence)

print("Random Walk:")

result = random_walk(karate_graph, start_node, num_steps)
print(result)

Random Walk:
[5, 0, 8, 32, 33, 32, 29, 23, 33, 29]


# Deep Walk Code

In [191]:
import numpy as np
import random
from gensim.models import Word2Vec
from torch.utils.data import Dataset
from functools import partial

__building dataset__

In [192]:
dataset = [random_walk(karate_graph, start_node=random.choice(list(karate_graph.nodes)), num_steps=20) for _ in range(100)]

In [193]:
for seq in dataset:
    print(seq)

[27, 23, 29, 32, 8, 30, 32, 31, 0, 21, 1, 21, 0, 11, 0, 17, 0, 21, 0, 10]
[6, 4, 6, 0, 12, 3, 12, 3, 7, 2, 1, 0, 31, 24, 27, 23, 33, 22, 32, 22]
[21, 1, 30, 32, 23, 25, 24, 31, 24, 25, 24, 27, 33, 15, 32, 14, 32, 15, 33, 20]
[2, 28, 31, 33, 28, 2, 28, 31, 28, 33, 15, 32, 20, 32, 22, 32, 22, 33, 23, 25]
[14, 32, 8, 33, 13, 2, 32, 33, 26, 29, 32, 30, 8, 32, 29, 33, 28, 31, 25, 23]
[33, 13, 3, 13, 2, 1, 0, 6, 16, 5, 16, 6, 16, 6, 16, 6, 5, 10, 4, 10]
[7, 0, 19, 33, 20, 33, 20, 32, 22, 33, 26, 33, 27, 23, 29, 23, 25, 31, 25, 23]
[33, 15, 33, 19, 33, 28, 33, 31, 0, 17, 0, 2, 28, 2, 28, 33, 19, 33, 31, 33]
[17, 0, 11, 0, 1, 0, 13, 1, 17, 1, 17, 1, 2, 9, 2, 8, 32, 30, 32, 30]
[13, 2, 13, 2, 1, 7, 3, 2, 32, 18, 33, 30, 32, 18, 33, 23, 25, 24, 31, 32]
[2, 7, 3, 7, 0, 17, 1, 17, 0, 31, 24, 27, 33, 30, 1, 2, 32, 33, 20, 33]
[8, 33, 19, 0, 21, 0, 5, 0, 7, 0, 12, 3, 1, 13, 2, 1, 13, 0, 6, 16]
[12, 3, 7, 0, 31, 24, 31, 25, 24, 31, 33, 14, 33, 23, 32, 14, 33, 29, 33, 26]
[23, 25, 31, 25, 31, 25, 23, 

__running DeepWalk__

In [194]:
deepwalk = Word2Vec(dataset, vector_size=300, epochs=100)

In [195]:
sorted(deepwalk.wv.index_to_key)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33]

# Node2Vec Code

__Random Walk Code for Node2Vec__

In [196]:

def node2vec_walk(G, start_node, num_steps, p, q):
    walk = [start_node]

    for _ in range(num_steps - 1):
        current_node = walk[-1]
        neighbors = list(G.neighbors(current_node))

        if len(neighbors) > 0:
            if len(walk) == 1:
                next_node = random.choice(neighbors)
            else:
                next_node = node2vec_weighted_choice(G, current_node, walk[-2], p, q)

            walk.append(next_node)
        else:
            break

    return walk

def node2vec_weighted_choice(G, current_node, previous_node, p, q):
    neighbors = list(G.neighbors(current_node))
    unnormalized_weights = []

    for neighbor in neighbors:
        if neighbor == previous_node:
            unnormalized_weights.append(1.0 / p)
        elif G.has_edge(current_node, neighbor):
            unnormalized_weights.append(1.0)
        else:
            unnormalized_weights.append(1.0 / q)

    norm_weights = [weight / sum(unnormalized_weights) for weight in unnormalized_weights]
    return random.choices(neighbors, weights=norm_weights)[0]

In [197]:
dataset2 = [node2vec_walk(karate_graph, start_node=random.choice(list(karate_graph.nodes)), num_steps=10, p=0.8, q=0.2) for _ in range(100)]

In [198]:
for i in dataset2:
    print(i)

[31, 24, 25, 23, 27, 23, 29, 33, 32, 20]
[12, 0, 3, 1, 3, 1, 13, 33, 18, 32]
[22, 33, 20, 32, 22, 32, 2, 9, 33, 20]
[15, 32, 15, 32, 29, 26, 33, 19, 1, 3]
[25, 24, 25, 24, 25, 31, 33, 13, 1, 13]
[11, 0, 5, 0, 8, 2, 28, 31, 0, 31]
[10, 4, 6, 16, 6, 16, 6, 5, 10, 0]
[27, 2, 9, 33, 27, 2, 7, 3, 7, 2]
[1, 2, 3, 0, 12, 3, 12, 3, 1, 13]
[6, 5, 16, 6, 16, 6, 5, 0, 7, 2]
[29, 33, 15, 33, 20, 33, 13, 33, 29, 26]
[7, 1, 19, 33, 32, 14, 33, 13, 33, 28]
[22, 32, 30, 32, 30, 32, 31, 0, 1, 2]
[29, 26, 29, 23, 27, 24, 27, 23, 32, 31]
[6, 4, 6, 0, 8, 0, 13, 3, 2, 27]
[32, 14, 33, 29, 26, 29, 33, 9, 2, 8]
[22, 33, 22, 32, 29, 32, 31, 33, 32, 33]
[5, 16, 5, 6, 5, 10, 0, 19, 33, 9]
[11, 0, 8, 0, 11, 0, 13, 1, 13, 1]
[31, 32, 2, 28, 33, 14, 32, 29, 26, 29]
[28, 33, 20, 32, 33, 13, 3, 0, 8, 2]
[27, 23, 33, 9, 33, 30, 32, 33, 32, 15]
[26, 29, 26, 33, 13, 2, 27, 33, 30, 8]
[0, 13, 0, 12, 0, 2, 0, 11, 0, 3]
[4, 0, 2, 13, 2, 27, 23, 29, 32, 2]
[20, 33, 8, 32, 33, 31, 33, 27, 24, 25]
[13, 1, 3, 2, 0, 19, 33, 30

In [199]:
node2vec = Word2Vec(dataset2, vector_size=200, epochs=100)

# LINE Code

__Alias Sampling__

In [200]:
import random
from decimal import *
import numpy as np
import collections
from tqdm import tqdm
import matplotlib.pyplot as plt


class VoseAlias:
    def __init__(self, dist):
        """
        (VoseAlias, dict) -> NoneType
        """
        self.dist = dist
        self.alias_initialisation()

    def alias_initialisation(self):
        """
        Construct probability and alias tables for the distribution.
        """
        # Initialise variables
        n = len(self.dist)
        self.table_prob = {}   # probability table
        self.table_alias = {}  # alias table
        scaled_prob = {}       # scaled probabilities
        small = []             # stack for probabilities smaller that 1
        large = []             # stack for probabilities greater than or equal to 1

        # Construct and sort the scaled probabilities into their appropriate stacks
        print("1/2. Building and sorting scaled probabilities for alias table...")
        for o, p in tqdm(self.dist.items()):
            scaled_prob[o] = Decimal(p) * n

            if scaled_prob[o] < 1:
                small.append(o)
            else:
                large.append(o)

        print("2/2. Building alias table...")
        # Construct the probability and alias tables
        while small and large:
            s = small.pop()
            l = large.pop()

            self.table_prob[s] = scaled_prob[s]
            self.table_alias[s] = l

            scaled_prob[l] = (scaled_prob[l] + scaled_prob[s]) - Decimal(1)

            if scaled_prob[l] < 1:
                small.append(l)
            else:
                large.append(l)

        # The remaining outcomes (of one stack) must have probability 1
        while large:
            self.table_prob[large.pop()] = Decimal(1)

        while small:
            self.table_prob[small.pop()] = Decimal(1)
        self.listprobs = list(self.table_prob)

    def alias_generation(self):
        """
        Yields a random outcome from the distribution.
        """
        # Determine which column of table_prob to inspect
        col = random.choice(self.listprobs)
        # Determine which outcome to pick in that column
        if self.table_prob[col] >= random.uniform(0, 1):
            return col
        else:
            return self.table_alias[col]

    def sample_n(self, size):
        """
        Yields a sample of size n from the distribution, and print the results to stdout.
        """
        for i in range(size):
            yield self.alias_generation()


def makeDist(graph: nx.Graph, power=0.75):

    edgedistdict = collections.defaultdict(int)
    nodedistdict = collections.defaultdict(int)

    weightsdict = collections.defaultdict(int)
    nodedegrees = collections.defaultdict(int)

    weightsum = 0
    negprobsum = 0

    nlines = 0

    maxindex = 0

    for edge in tqdm(graph.edges(data=True), total=nlines):
        node1, node2, weight = edge[0], edge[1], edge[2]["weight"]

        edgedistdict[tuple([node1, node2])] = weight
        nodedistdict[node1] += weight

        weightsdict[tuple([node1, node2])] = weight
        nodedegrees[node1] += weight

        weightsum += weight
        negprobsum += np.power(weight, power)

        if node1 > maxindex:
            maxindex = node1
        elif node2 > maxindex:
            maxindex = node2

    for node, outdegree in nodedistdict.items():
        nodedistdict[node] = np.power(outdegree, power) / negprobsum

    for edge, weight in edgedistdict.items():
        edgedistdict[edge] = weight / weightsum

    return edgedistdict, nodedistdict, weightsdict, nodedegrees, maxindex


def negSampleBatch(sourcenode, targetnode, negsamplesize, weights,
                   nodedegrees, nodesaliassampler, t=10e-3):
    """
    For generating negative samples.
    """
    negsamples = 0
    while negsamples < negsamplesize:
        samplednode = nodesaliassampler.sample_n(1)
        if (samplednode == sourcenode) or (samplednode == targetnode):
            continue
        else:
            negsamples += 1
            yield samplednode


def makeData(samplededges, negsamplesize, weights, nodedegrees, nodesaliassampler):
    for e in samplededges:
        sourcenode, targetnode = e[0], e[1]
        negnodes = []
        for negsample in negSampleBatch(sourcenode, targetnode, negsamplesize,
                                        weights, nodedegrees, nodesaliassampler):
            for node in negsample:
                negnodes.append(node)
        yield [e[0], e[1]] + negnodes

__LINE Model code__

In [201]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class LINE(nn.Module):
    def __init__(self, size, embed_dim=128, order=1):
        super(LINE, self).__init__()

        assert order in [1, 2], print("Order should either be int(1) or int(2)")

        self.embed_dim = embed_dim
        self.order = order
        self.nodes_embeddings = nn.Embedding(size, embed_dim)

        if order == 2:
            self.contextnodes_embeddings = nn.Embedding(size, embed_dim)
            # Initialization
            self.contextnodes_embeddings.weight.data = self.contextnodes_embeddings.weight.data.uniform_(
                -.5, .5) / embed_dim

        # Initialization
        self.nodes_embeddings.weight.data = self.nodes_embeddings.weight.data.uniform_(
            -.5, .5) / embed_dim

    def forward(self, v_i, v_j, negsamples, device):

        v_i = self.nodes_embeddings(v_i).to(device)

        if self.order == 2:
            v_j = self.contextnodes_embeddings(v_j).to(device)
            negativenodes = -self.contextnodes_embeddings(negsamples).to(device)

        else:
            v_j = self.nodes_embeddings(v_j).to(device)
            negativenodes = -self.nodes_embeddings(negsamples).to(device)

        mulpositivebatch = torch.mul(v_i, v_j)
        positivebatch = F.logsigmoid(torch.sum(mulpositivebatch, dim=1))

        mulnegativebatch = torch.mul(v_i.view(len(v_i), 1, self.embed_dim), negativenodes)
        negativebatch = torch.sum(
            F.logsigmoid(
                torch.sum(mulnegativebatch, dim=2)
            ),
            dim=1)
        loss = positivebatch + negativebatch
        return -torch.mean(loss)

In [202]:
for edge in karate_graph.edges():
    karate_graph.edges[edge]["weight"] = 1
    

In [203]:
from tqdm import trange
import torch
import torch.optim as optim
import sys
import pickle

args = {
    "epochs": 100,
    "order": 2,
    "num_neg": 5,
    "dim": 128,
    "batch_size": 10,
    "lr": 0.025,
    "neg_power": 0.75
}
# Create dict of distribution when opening file
edgedistdict, nodedistdict, weights, nodedegrees, maxindex = makeDist(
    karate_graph, args["neg_power"])

edgesaliassampler = VoseAlias(edgedistdict)
nodesaliassampler = VoseAlias(nodedistdict)

batchrange = int(len(edgedistdict) / args["batch_size"])
print(maxindex)
line = LINE(maxindex + 1, embed_dim=args["dim"], order=args["order"])

opt = optim.SGD(line.parameters(), lr=args["lr"],
                    momentum=0.9, nesterov=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lossdata = {"it": [], "loss": []}
it = 0

print("\nTraining on {}...\n".format(device))
for epoch in range(args["epochs"]):
    for b in trange(batchrange, desc=f"Epoch {epoch}"):
        samplededges = edgesaliassampler.sample_n(args["batch_size"])
        batch = list(makeData(samplededges, args["num_neg"], weights, nodedegrees,
                                  nodesaliassampler))
        batch = torch.LongTensor(batch)
        v_i = batch[:, 0]
        v_j = batch[:, 1]
        negsamples = batch[:, 2:]
        line.zero_grad()
        loss = line(v_i, v_j, negsamples, device)
        loss.backward()
        opt.step()

        lossdata["loss"].append(loss.item())
        lossdata["it"].append(it)
        it += 1

156it [00:00, ?it/s]


1/2. Building and sorting scaled probabilities for alias table...


100%|██████████| 156/156 [00:00<?, ?it/s]


2/2. Building alias table...
1/2. Building and sorting scaled probabilities for alias table...


100%|██████████| 34/34 [00:00<?, ?it/s]


2/2. Building alias table...
33

Training on cuda:0...


Epoch 99: 100%|██████████| 15/15 [00:00<00:00, 960.12it/s]
